In [3]:
from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import MinMaxScaler
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
import numpy as np
import random
from pyspark.sql.functions import udf
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline





In [5]:
spark = SparkSession.builder.appName("Assign1").getOrCreate()    


sparkDF = spark.read.csv("MSD.txt",header=False,sep=",",inferSchema=True)


### Part 1

In [6]:
sparkDF.count()

515345

In [12]:
sparkDF.head(40)

[Row(_c0=2001, _c1=49.94357, _c2=21.47114, _c3=73.0775, _c4=8.74861, _c5=-17.40628, _c6=-13.09905, _c7=-25.01202, _c8=-12.23257, _c9=7.83089, _c10=-2.46783, _c11=3.32136, _c12=-2.31521, _c13=10.20556, _c14=611.10913, _c15=951.0896, _c16=698.11428, _c17=408.98485, _c18=383.70912, _c19=326.51512, _c20=238.11327, _c21=251.42414, _c22=187.17351, _c23=100.42652, _c24=179.19498, _c25=-8.41558, _c26=-317.87038, _c27=95.86266, _c28=48.10259, _c29=-95.66303, _c30=-18.06215, _c31=1.96984, _c32=34.42438, _c33=11.7267, _c34=1.3679, _c35=7.79444, _c36=-0.36994, _c37=-133.67852, _c38=-83.26165, _c39=-37.29765, _c40=73.04667, _c41=-37.36684, _c42=-3.13853, _c43=-24.21531, _c44=-13.23066, _c45=15.93809, _c46=-18.60478, _c47=82.15479, _c48=240.5798, _c49=-10.29407, _c50=31.58431, _c51=-25.38187, _c52=-3.90772, _c53=13.29258, _c54=41.5506, _c55=-7.26272, _c56=-21.00863, _c57=105.50848, _c58=64.29856, _c59=26.08481, _c60=-44.5911, _c61=-8.30657, _c62=7.93706, _c63=-10.7366, _c64=-95.44766, _c65=-82.03307

### Part 2
Reference
https://www.datacamp.com/community/tutorials/apache-spark-tutorial-machine-learning

In [15]:
# Import `DenseVector`
from pyspark.ml.linalg import DenseVector

# Create data as target and condense all the row values into dense vector
## df.rdd conversts each row as one entry and map iterates through the row
input_data = sparkDF.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# Replace `df` with the new DataFrame
condensed_df = spark.createDataFrame(input_data, ["Year", "features"])

In [19]:
# Import `StandardScaler` 
from pyspark.ml.feature import StandardScaler

# Initialize the `standardScaler`
MMScaler = MinMaxScaler(inputCol="features", outputCol="features_scaled")

# Fit and transform the DataFrame to the scaler
scaled_df = MMScaler.fit(condensed_df).transform(condensed_df)


# Inspect the result
scaled_df.take(2)

[Row(Year=2001, features=DenseVector([49.9436, 21.4711, 73.0775, 8.7486, -17.4063, -13.099, -25.012, -12.2326, 7.8309, -2.4678, 3.3214, -2.3152, 10.2056, 611.1091, 951.0896, 698.1143, 408.9848, 383.7091, 326.5151, 238.1133, 251.4241, 187.1735, 100.4265, 179.195, -8.4156, -317.8704, 95.8627, 48.1026, -95.663, -18.0622, 1.9698, 34.4244, 11.7267, 1.3679, 7.7944, -0.3699, -133.6785, -83.2616, -37.2976, 73.0467, -37.3668, -3.1385, -24.2153, -13.2307, 15.9381, -18.6048, 82.1548, 240.5798, -10.2941, 31.5843, -25.3819, -3.9077, 13.2926, 41.5506, -7.2627, -21.0086, 105.5085, 64.2986, 26.0848, -44.5911, -8.3066, 7.9371, -10.7366, -95.4477, -82.0331, -35.5919, 4.6952, 70.9563, 28.0914, 6.0202, -37.1377, -41.1245, -8.4082, 7.1988, -8.6018, -5.9086, -12.3244, 14.6873, -54.3212, 40.1479, 13.0162, -54.4055, 58.9937, 15.3734, 1.1114, -23.0879, 68.408, -1.8222, -27.4635, 2.2633]), features_scaled=DenseVector([0.8003, 0.4972, 0.5996, 0.3325, 0.3706, 0.277, 0.4526, 0.3025, 0.4924, 0.384, 0.4629, 0.5041, 

In [26]:

minYear = scaled_df.agg({"Year": "min"}).collect()[0][0]



In [27]:
minYear

1922

In [29]:
scaled_df = scaled_df.withColumn("Year_shifted",col('Year')- minYear )

In [31]:
scaled_df.take(2)

[Row(Year=2001, features=DenseVector([49.9436, 21.4711, 73.0775, 8.7486, -17.4063, -13.099, -25.012, -12.2326, 7.8309, -2.4678, 3.3214, -2.3152, 10.2056, 611.1091, 951.0896, 698.1143, 408.9848, 383.7091, 326.5151, 238.1133, 251.4241, 187.1735, 100.4265, 179.195, -8.4156, -317.8704, 95.8627, 48.1026, -95.663, -18.0622, 1.9698, 34.4244, 11.7267, 1.3679, 7.7944, -0.3699, -133.6785, -83.2616, -37.2976, 73.0467, -37.3668, -3.1385, -24.2153, -13.2307, 15.9381, -18.6048, 82.1548, 240.5798, -10.2941, 31.5843, -25.3819, -3.9077, 13.2926, 41.5506, -7.2627, -21.0086, 105.5085, 64.2986, 26.0848, -44.5911, -8.3066, 7.9371, -10.7366, -95.4477, -82.0331, -35.5919, 4.6952, 70.9563, 28.0914, 6.0202, -37.1377, -41.1245, -8.4082, 7.1988, -8.6018, -5.9086, -12.3244, 14.6873, -54.3212, 40.1479, 13.0162, -54.4055, 58.9937, 15.3734, 1.1114, -23.0879, 68.408, -1.8222, -27.4635, 2.2633]), features_scaled=DenseVector([0.8003, 0.4972, 0.5996, 0.3325, 0.3706, 0.277, 0.4526, 0.3025, 0.4924, 0.384, 0.4629, 0.5041, 

#### Test Train and Validation split

In [34]:
train, validation , test = scaled_df.randomSplit([0.6, 0.2, 0.2],seed = 42)

Create a baseline model where we always provide the same prediction irrespective of our input. (Use training data)

In [38]:
yearFreq = train.groupBy('Year').count()

In [42]:
modeOfYear = yearFreq.orderBy('count',ascending=False).collect()[0][0]

In [56]:
prediction = test.rdd.map(lambda x :modeOfYear) ## predict mode for all the examples
actualLabels = test.select("Year").rdd.map(lambda x : x[0])

In [58]:
prediction.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 59.0 failed 1 times, most recent failure: Lost task 1.0 in stage 59.0 (TID 1571, localhost, executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(Unknown Source)
	at java.net.SocketOutputStream.write(Unknown Source)
	at java.io.BufferedOutputStream.write(Unknown Source)
	at java.io.DataOutputStream.write(Unknown Source)
	at java.io.FilterOutputStream.write(Unknown Source)
	at org.apache.spark.api.python.PythonRDD$.org$apache$spark$api$python$PythonRDD$$write$1(PythonRDD.scala:212)
	at org.apache.spark.api.python.PythonRDD$$anonfun$writeIteratorToStream$1.apply(PythonRDD.scala:224)
	at org.apache.spark.api.python.PythonRDD$$anonfun$writeIteratorToStream$1.